In [ ]:
get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')
from prepare_cloth import *
from anchor_check import *
# import sys
# default_stdout = sys.stdout
# sys.stdout = open('../log/log_prepare_cloth.txt', 'w')

# Crop and collect images from the anchor detection results

In [ ]:
anchor_dict = pickle.load(open('../data/anchor_dict_all.pkl', 'rb'))

In [ ]:
from anchor_detect import *
anchor_dict_clean = clean_anchor_dict(anchor_dict, people_dict)
pickle.dump(anchor_dict_clean, open('../data/anchor_dict_all_clean.pkl', "wb" ), protocol=2)  

In [ ]:
# remove existed video
res_dict = pickle.load(open('../data/cloth/cloth_manifest.pkl', 'rb'))
for p in res_dict_clean:
    if p[0] in anchor_dict:
        del anchor_dict[p[0]]
        

In [ ]:
solve_parallel_memory(anchor_dict, '../data/cloth/cloth_half3_manifest.pkl', 64)

# Collect tmp result

In [ ]:
res_dict2 = []
nthread = 32
tmp_dict_list = []
for i in range(nthread):
    tmp_dict_list.append('../tmp/cloth_dict_' + str(i) + '.pkl')

for path in tmp_dict_list:
    dict_file = Path(path)
    if not dict_file.is_file():
        continue
    res_dict_tmp = pickle.load(open(path, "rb" ))
    res_dict2.extend(res_dict_tmp)

pickle.dump(res_dict2, open('../data/cloth/cloth_half0_manifest2.pkl', "wb" ), protocol=2)  

In [ ]:
res_dict = pickle.load(open('../data/cloth/cloth_manifest.pkl', "rb" ))
res_dict2 = pickle.load(open('../data/cloth/cloth_half3_manifest.pkl', "rb" ))
res_dict.extend(res_dict2)
print(len(res_dict))
# pickle.dump(res_dict, open('../data/cloth/cloth_manifest.pkl', "wb" ), protocol=2)  

In [ ]:
# clean repeated result
repeate = {}
repeate_video = {}
res_dict_clean = []
cnt = 0
for res in res_dict2:
    repeate_video[res[0]] = 0
    if res[2] in repeate:
        print(res[2], res[3], repeate[res[2]])
        cnt += 1
        pass
    else: 
        repeate[res[2]] = res[3]
        res_dict_clean.append(res)
print(len(res_dict_clean))
print(len(res_dict2))
print(len(repeate_video))

In [ ]:
pickle.dump(res_dict_clean, open('../data/cloth/cloth_manifest.pkl', 'wb'), protocol=2)

# Recollect bbox for cloth_label dataset

In [ ]:
cloth_label = pickle.load(open('../data/cloth/newsAnchor_train_manifest.pkl', 'rb'))

In [ ]:
video = {}
for label in cloth_label:
    video[label[0][0]] = 0
video_list = sorted(video)
print(len(video))

In [ ]:
anchor_dict_tmp = {video: anchor_dict[video] for video in video_list}

In [ ]:
solve_parallel_memory(anchor_dict_tmp, '../tmp/newsAnchor_train_manifest_makeup.pkl', 32)

In [ ]:
cloth_label_makup = pickle.load(open('../tmp/newsAnchor_train_manifest_makeup.pkl', 'rb'))

In [ ]:
for group in cloth_label:
    for label in group:
        find = False
        for makeup in cloth_label_makup:
            if makeup[2] == label[2]:
                label.append(makeup[3])
                find = True
                break
        if not find:
            print(label[0])

In [ ]:
# print(cloth_label[0][0])
test = cloth_label[35][4]
img = cv2.imread('../data/cloth/cloth_label/' + test[2])
x1 = test[4]['bbox_x1']
y1 = test[4]['bbox_y1']
x2 = test[4]['bbox_x2']
y2 = test[4]['bbox_y2']
print(x1, y1, x2, y2)
cv2.rectangle(img, (x1, y1), (x2, y2), color=(0,0,255), thickness=3)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

# Reconstruct cloth_all_infer (major vote for cloth attribute, include feature_vec, include gendre)

In [ ]:
anchor_dict = pickle.load(open('../data/anchor_dict_all.pkl', 'rb'))
# gender_dict = pickle.load(open('../data/cloth/gender_dict.pkl', 'rb'))
# cloth_infer = pickle.load(open('../data/cloth/cloth_all_infer.pkl', "rb" ))
# feature_dict = pickle.load(open('../data/cloth/cloth_all_feature.pkl', "rb" ))

In [ ]:
def vote_gender_for_anchor(anchor_person, gender_list):
    gender = {'F': 0, 'M': 0, 'U': 0}
    for p in anchor_person:
        candidate = gender_list[p['fid']]
        for c in candidate:
            if (p['bbox']['bbox_x1'] - c[0]['bbox_x1'] + p['bbox']['bbox_y1'] - c[0]['bbox_y1']) < 1e-5:
                gender[c[1]] += 1
                break
    if gender['F'] + gender['M'] == 0:
        print('bad gender')
    if gender['F'] > gender['M']:
        return 'F'
    else:
        return 'M'

In [ ]:
## create structure of [[manifest per anchor]] 
pid = -1
current_video = ''
current_anchor = 0
cloth_infer_group = []
for idx, p in enumerate(cloth_infer):
    video = p[0]
    anchor_id = p[1]
    if video != current_video or anchor_id != current_anchor:
        cloth_infer_group.append([])
        current_video = video
        current_anchor = anchor_id
    p.append(feature_dict[idx][1])
    cloth_infer_group[-1].append(p)

In [ ]:
cloth_infer_group[0][0]

In [ ]:
NUM_VOTE = 5
MAX_CLASS = 20
    
cloth_dict = []
for pid, p_group in enumerate(cloth_infer_group):
    video = p_group[0][0]
    anchor_id = p_group[0][1]
    ## major vote for cloth attributes
    votes = []
    num_vote = min(len(p_group), NUM_VOTE)
    for i in range(num_vote):
        votes.append(p_group[i][4])
    votes = np.array(votes).transpose()
    major_vote = []
    for idx, vote in enumerate(votes):
        cnt = np.zeros(MAX_CLASS)
        for v in vote:
            cnt[int(v)] += 1
        major_vote.append(np.argmax(cnt))
    ## check bbox to find gender
    gender = vote_gender_for_anchor(anchor_dict[video][anchor_id], gender_dict[video])
    ## stitch for new item
    new_p = p_group[0][:4] + [gender, major_vote, p_group[0][-1]]
    cloth_dict.append(new_p)
    

In [ ]:
# len(cloth_dict)
pickle.dump(cloth_dict, open('../data/cloth/newsAnchor_all_data.pkl', 'wb'), protocol=2)

# random sample image from group

In [ ]:
# cloth_dict = pickle.load(open('../data/cloth/newsAnchor_all_data_first.pkl', 'rb'))
# people_dict = pickle.load(open('../data/people_dict_half0.pkl', 'rb'))
trans = {'video': 0, 'anchor_id': 1, 'path': 2, 'detail': 3, 'bbox': 4, 'gender':5, 'attributes': 6, 'feature': 7}
hair_color_dict = pickle.load(open('../data/cloth/hair_color_NN.pkl', 'rb'))

In [ ]:
print(len(hair_color_dict), len(cloth_dict), len(cloth_infer_group))

In [ ]:
def clean_anchor(video, anchor_person, people_dict):
    FACE_SIM_THRESH = 0.9
    OTHER_SHOW_THRESH = 85
    date, station, show = get_detail_from_video_name(video)
    ## find cluster center
    for p in anchor_person:
        if p['sim'] == 0:
            anchor = p
            break
    ## find this anchor in other shows
    cnt = 0
    for show_check, people_list in people_dict.items():
        if show != show_check:
            for p in people_list:
                dist = np.linalg.norm(p['feature'] - anchor['feature'])
                if dist < FACE_SIM_THRESH:
                    cnt += 1
                    break
    if cnt < OTHER_SHOW_THRESH:
        return True
    else:
        print(video, cnt)
        return False


In [ ]:
# remove Trump & Cliton
cloth_dict_new = []
for idx, cloth in enumerate(cloth_dict):
    video = cloth[trans['video']]
    anchor_id = cloth[trans['anchor_id']]
    if clean_anchor(video, anchor_dict[video][anchor_id], people_dict):
        cloth_dict_new.append(cloth)
    
    if idx % 100 == 0:
        print(idx)

In [ ]:
# random sample & add hair color from NN
cloth_dict_new = []
np.random.seed(7777)
for idx, cloth in enumerate(cloth_dict):
    video = cloth[trans['video']]
    anchor_id = cloth[trans['anchor_id']]
    group = cloth_infer_group[idx]
    choice = np.random.choice(len(group), 1)[0]
    
    assert(video == group[0][0] and video == hair_color_dict[idx][0])
    assert(anchor_id == group[0][1] and anchor_id == hair_color_dict[idx][1])
    
    sample = group[choice]
    path = sample[2]
    feature = sample[5]
    bbox = sample[3]
    attributes = copy.deepcopy(cloth[trans['attributes']])
    attributes.append(hair_color_dict[idx][2])
    
    new_cloth = copy.deepcopy(cloth)
    new_cloth[trans['path']] = path
    new_cloth[trans['feature']] = feature
    new_cloth[trans['attributes']] = attributes
    new_cloth[trans['bbox']] = bbox
    cloth_dict_new.append(new_cloth)
    

In [ ]:
pickle.dump(cloth_dict_new, open('../data/cloth/newsAnchor_all_data_random.pkl', 'wb'), protocol=2)